In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
import pdb

Function *myCooDescent* solves Lasso problem with  given inputs: <br />
w0: initial condition for offset term <br />
w: initial guess for coefficients<br />
X,y: data <br />
lyambda: Lasso parametr<br />

In [38]:
def myCooDescent(w0,w,y,X,lyambda):
    N,d=X.shape
#     print d,N
    wOld=w #keep for computations
    wNew=wOld 
    kmax=50 #max number of iterations
    eps=10**(-3) # for convergence
    w0old=w0
    
    for kk in range(kmax):
#         pdb.set_trace()
        errorOld=np.square(np.linalg.norm(y-X*wOld-w0old, ord=2))+lyambda*np.linalg.norm(wOld, ord=1)
        
        yhat=X*wOld+w0old
        w0new=np.sum(y-yhat+w0old)/N
        yhat=yhat-w0old+w0new
    
        for jj in range(d):
#             pdb.set_trace()
            start, end = X.indptr[jj], X.indptr[jj+1]
            aj=2*np.sum(np.square(X.data[start:end]))
            
            cj=2*np.sum(X.data[start:end]*(y[X.indices[start:end]]-\
                                               yhat[X.indices[start:end]]+X.data[start:end]*wOld[jj]))
            if cj+lyambda<0:
                wNew[jj]=(cj+lyambda)/aj
            elif cj-lyambda>0:
                wNew[jj]=(cj-lyambda)/aj
            else:
                wNew[jj]=0
                
        if np.linalg.norm(wNew-wOld,ord=np.inf)<eps:
#             print('converged')
            return (w0,wNew)
        else:
            wOld=wNew
            w0old=w0new
            
        errorNew=np.square(np.linalg.norm(y-X*wNew-w0, ord=2))+lyambda*np.linalg.norm(wNew, ord=1)
        if errorOld-errorNew<10**(-3):
            print('error grows', jj)
                
    if kk==kmax-1:
        print('reached max iter')
        return (w0, wNew)
            

###Test on synthetic data:

In [39]:
N, d, k, sigma=50,75,5,1
X=np.random.randn(N,d)
r=np.random.randint(2, size=k)
r[r==0]=-1
r=r*10;
zer=np.zeros(d-k)
wStar=np.concatenate((r,zer))
noise =np.sqrt(sigma) * np.random.randn(N)
Xsparse=sp.csc_matrix(X)
XT=X.T
y=Xsparse*wStar+noise

Compute initial conditions

In [46]:
w0=np.random.randn(1) # random initial guess
wGuess=np.random.randn(d) # random initial guess
##Calculate Lambda_max
lyambda=2*np.linalg.norm(np.dot(XT,y-np.mean(y)), ord=np.inf)


In [47]:
nloop=5
# precision=np.zeros(nloop)
# recall=np.zeros(nloop)
for i in range(30):
    w0,wGuess=myCooDescent(w0,wGuess,y,Xsparse,lyambda)
    lyambda=0.9*lyambda
#     print(w0, wGuess[0:10])

In [48]:
wGuess


array([ 9.59488434, -9.37767555,  9.57189766, -9.37755019,  9.56669092,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.10602876,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [10]:
wGuess[0:15]

array([  8.20888890e+24,  -9.43199922e+23,   1.11429795e+25,
        -6.97412193e+24,   1.13066913e+25,   3.40052893e+24,
         1.23909478e+25,   1.88313468e+24,  -5.17736739e+24,
         1.85927772e+23,  -4.49456060e+24,   6.77347273e+24,
        -1.25677429e+24,  -8.96755066e+24,  -7.85347958e+24])